In [1]:
import pandas as pd 
import numpy as np

col_names = ['COD_ID','DIST','LGRD','BRR','MUNICIPIO','ESTADO',
            'UF','COD_UF','CEP','CNAE','GRU_TAR','SIT_ATIV',
            'CLAS_SUB','DAT_CON']

bdgd = pd.read_csv('./Bases/Ucs_UCMT.csv',
                   dtype={colname:str for colname in col_names},sep=';',
                   encoding='latin-1')

/tmp/ipykernel_17183/1247011084.py:8: DtypeWarning: Columns (1,3,4,5,6,74) have mixed types. Specify dtype option on import or set low_memory=False.
  bdgd = pd.read_csv('./Bases/Ucs_UCMT.csv',


In [2]:
#Verificando se existe colunas com mais de um types
def vefica_types(dataframe):
    
    verifica = [print(colname,str(types) + '-Corrigir')
            if types > 1 else None
            for colname,types in dataframe.applymap(type).nunique().items()]

In [3]:
def ibge_completo():
    
    municipios_ibge = pd.read_excel('./Bases/RELATORIO_DTB_BRASIL_DISTRITO.xls',
    usecols=['Código Município Completo','UF','Nome_Município'],
    dtype={'Código Município Completo':int,
    'UF':int,
    'Nome_Município':str})

    ibge_estados = pd.read_excel('./Bases/Tabela_Estados_IBGE.xlsx',
    dtype={'Código da UF':int,
    'Estado':str})

    #Usar loop para nomear string as colunas *****

    #Mudando Nome da Coluna do IBGE e Estados
    municipios_ibge\
        .rename(columns={'Código Município Completo':'MUN',
        'Nome_Município':'MUNICIPIO','UF':'COD_UF'},inplace=True)

    ibge_estados\
        .rename(columns={'Código da UF':'COD_UF','Estado':"ESTADO"},inplace=True)


    #Deixando Municipios e Estado em extenso em letra maiscula
    municipios_ibge.MUNICIPIO = municipios_ibge\
        .MUNICIPIO.str.upper()

    ibge_estados.ESTADO = ibge_estados\
        .ESTADO.str.upper()

    #Cruzando com IBGE com siglas de Municipio 
    ibge_completo = municipios_ibge.merge(ibge_estados,on='COD_UF')

    #Removendo linhas duplicadas do datafrmae
    ibge_completo.drop_duplicates(inplace=True)

    return ibge_completo
    
ibge_completo = ibge_completo()

In [4]:
print(f"bdgd{bdgd.shape} | ibge {ibge_completo.shape}")

bdgd(292989, 78) | ibge (5570, 5)


In [5]:
#Cruzando codigo_mun bdgd com ibge_completo
dataframe_completo = bdgd.merge(ibge_completo,on='MUN',
how='left',copy=True)  

print(f"Cruzamento{dataframe_completo.shape} ")

Cruzamento(292989, 82) 


In [6]:
df_consolidado = dataframe_completo[:] ## ** Melhorar código depois

In [7]:
df_consolidado['ENE_MED'] = df_consolidado.loc[:, 'ENE_01':'ENE_12'].mean(axis=1)
df_consolidado['DEM_MAX'] = df_consolidado.loc[:, 'DEM_01':'DEM_12'].max(axis=1)
df_consolidado['DIC'] = df_consolidado.loc [:,'DIC_01':'DIC_12'].sum(axis=1)
df_consolidado['FIC'] = df_consolidado.loc [:,'FIC_01':'FIC_12'].sum(axis=1)

/tmp/ipykernel_17183/2011574496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado['ENE_MED'] = df_consolidado.loc[:, 'ENE_01':'ENE_12'].mean(axis=1)
/tmp/ipykernel_17183/2011574496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado['DEM_MAX'] = df_consolidado.loc[:, 'DEM_01':'DEM_12'].max(axis=1)
/tmp/ipykernel_17183/2011574496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [8]:
#Limpando Colunas desnecessárias

#Excluir Colunas de Energia mês a mês Demanda e Consumo


#Demanda
df_consolidado.drop(['DEM_01', 'DEM_02', 'DEM_03', 'DEM_04', 'DEM_05', 'DEM_06', 'DEM_07',
'DEM_08', 'DEM_09', 'DEM_10','DEM_11', 'DEM_12'],
axis= 1 ,inplace=True)

#Consumo
df_consolidado.drop(['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04',
 'ENE_05', 'ENE_06', 'ENE_07', 'ENE_08', 'ENE_09', 'ENE_10', 'ENE_11',
                     'ENE_12'],axis= 1,inplace=True)


df_consolidado.drop(['DESCR'],axis= 1,inplace=True)


df_consolidado.drop(['DIC_01', 'DIC_02', 'DIC_03', 'DIC_04', 'DIC_05','DIC_06', 'DIC_07', 
'DIC_08', 'DIC_09', 'DIC_10', 'DIC_11', 'DIC_12'],axis= 1,inplace=True)


df_consolidado.drop(['FIC_01', 'FIC_02', 'FIC_03', 'FIC_04', 'FIC_05', 'FIC_06', 'FIC_07',
'FIC_08', 'FIC_09', 'FIC_10', 'FIC_11', 'FIC_12'],axis=1,inplace=True)


/tmp/ipykernel_17183/1790195318.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.drop(['DEM_01', 'DEM_02', 'DEM_03', 'DEM_04', 'DEM_05', 'DEM_06', 'DEM_07',
/tmp/ipykernel_17183/1790195318.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.drop(['ENE_01', 'ENE_02', 'ENE_03', 'ENE_04',
/tmp/ipykernel_17183/1790195318.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.drop(['DESCR'],axi

In [9]:
#Renomear a CTAT_CTMT para esse nome tanto no AT e MT
df_consolidado.rename(columns={'CTAT':'CT_AT_MT'},inplace=True)

/tmp/ipykernel_17183/2530880738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consolidado.rename(columns={'CTAT':'CT_AT_MT'},inplace=True)


In [10]:
df_consolidado.keys()

Index(['COD_ID', 'PN_CON', 'DIST', 'PAC', 'CTMT', 'UNI_TR_AT', 'SUB', 'CONJ',
       'MUN', 'CEG_GD', 'LGRD', 'BRR', 'CEP', 'CLAS_SUB', 'CNAE', 'TIP_CC',
       'FAS_CON', 'GRU_TEN', 'TEN_FORN', 'GRU_TAR', 'SIT_ATIV', 'DAT_CON',
       'CAR_INST', 'LIV', 'ARE_LOC', 'SEMRED', 'DEM_CONT', 'xcoord', 'ycoord',
       'COD_UF', 'MUNICIPIO', 'UF', 'ESTADO', 'ENE_MED', 'DEM_MAX', 'DIC',
       'FIC'],
      dtype='object')

In [1]:
# Connect SqlAlchemy
from sqlalchemy import (MetaData,Table,String,
Integer,Float,Column,insert,Numeric,inspect,select,insert)
from common.base import connection_oracle_sqlalchemy

engine= connection_oracle_sqlalchemy()
metadata= MetaData()
connection = engine.connect()
insp = inspect(engine)

In [2]:
#Criando Tabela Oracle 

bdgd_mt = Table('BDGD_MT',metadata,
         Column('COD_ID', String(255)),
         Column('DIST', String(255)),
         Column('LGRD', String(255)),
         Column('BRR', String(255)),
         Column('MUNICIPIO', String(255)),       
         Column('ESTADO', String(255)),
         Column('UF', String(255)),
         Column('COD_UF', String(10)),
         Column('MUN', Numeric),
         Column('CEP', String(255)),
         Column('CNAE', String(255)),
         Column('ENE_MED', Float),
         Column('DEM_MAX', Float),
         Column('DEM_CONT',Float),
         Column('LIV', Numeric),
         Column('GRU_TAR',String(255)),                   
         Column('SIT_ATIV', String(255)),
         Column('CLAS_SUB', String(255)),
         Column('DAT_CON', String(255)))
                               

metadata.create_all(engine)

In [24]:
#Load Table
bdgd_mt = Table('BDGD_MT',metadata,
         autoload=True,
         autoload_with=engine)

In [13]:
# Loc columns to commit 
df_bdgd_mt = df_consolidado\
    .loc[:,['COD_ID','DIST','LGRD','BRR','MUNICIPIO','ESTADO',
            'UF','COD_UF','MUN','CEP','CNAE','ENE_MED',
            'DEM_MAX','DEM_CONT','LIV','GRU_TAR','SIT_ATIV',
            'CLAS_SUB','DAT_CON']].copy()

In [16]:
#Verificar todos mixtypes
vefica_types(df_bdgd_mt)

In [15]:
#MixType Correct
df_bdgd_mt.BRR = df_bdgd_mt.BRR.astype(str)
df_bdgd_mt.MUNICIPIO = df_bdgd_mt.MUNICIPIO.astype(str)
df_bdgd_mt.ESTADO = df_bdgd_mt.ESTADO.astype(str)
df_bdgd_mt.UF = df_bdgd_mt.UF.astype(str)
df_bdgd_mt.CNAE = df_bdgd_mt.CNAE.astype(str)

In [17]:
#Tratment Columns 

df_bdgd_mt.isna().value_counts()
df_bdgd_mt.fillna(0,inplace=True)

df_bdgd_mt.COD_UF = df_bdgd_mt.COD_UF.astype(str)
df_bdgd_mt.COD_UF = df_bdgd_mt.COD_UF.str.replace('.0','',regex=True)

In [18]:
#Tranformando List-Dict to SqlAlchemy
values_list =[{'COD_ID':row.COD_ID, 'DIST':row.DIST, 'LGRD':row.LGRD, 'BRR':row.BRR,
  'MUNICIPIO':row.MUNICIPIO, 'ESTADO':row.ESTADO, 'UF':row.UF,
  'COD_UF':row.COD_UF, 'MUN':row.MUN, 'CEP':row.CEP,
  'CNAE':row.CNAE, 'ENE_MED':row.ENE_MED, 'DEM_MAX':row.DEM_MAX,
  'LIV':row.LIV, 'GRU_TAR':row.GRU_TAR,'SIT_ATIV':row.SIT_ATIV,
  'CLAS_SUB':row.CLAS_SUB,'DAT_CON':row.DAT_CON} for row in df_bdgd_mt.itertuples()] 

In [19]:
#Commit Oracle - SQL 
stmt = insert(bdgd_mt)
results_proxy = connection.execute(stmt,values_list,batcherrors=True)
print(results_proxy.rowcount)

292989
